# Data retrival

In [2]:
%load_ext autoreload
%autoreload 2
import myfunctions as f
f.today()

'20180314'

In [3]:
# python version
import sys
print "system_info: %s"%sys.version
# current working directory
import os
print "path_info: %s"%os.getcwd() 

system_info: 2.7.14 |Anaconda custom (64-bit)| (default, Oct 27 2017, 18:21:12) 
[GCC 7.2.0]
path_info: /home/yueliu/Desktop/workspace_yue/Documentation_201712


In [4]:
# setup elastic search
f.initialise_es(1)
foodpanda, deliveroo, wte, websites = f.delivery_para()

{
  "name" : "star7",
  "cluster_name" : "star",
  "cluster_uuid" : "VDcV69ThThyb00sd6PgeMA",
  "version" : {
    "number" : "2.4.5",
    "build_hash" : "c849dd13904f53e63e88efc33b2ceeda0b6a1276",
    "build_timestamp" : "2017-04-24T16:18:17Z",
    "build_snapshot" : false,
    "lucene_version" : "5.5.4"
  },
  "tagline" : "You Know, for Search"
}



** Retriving all food items **

In [5]:
doc_type = "food"

In [6]:
%%time
website = websites[0]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 2h 32min 15s, sys: 10min 4s, total: 2h 42min 20s
Wall time: 2h 43min 3s


In [7]:
%%time
website = websites[1]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 1h 31min 17s, sys: 5min 15s, total: 1h 36min 32s
Wall time: 1h 37min 16s


In [8]:
%%time
website = websites[2]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

CPU times: user 14min 50s, sys: 30.9 s, total: 15min 21s
Wall time: 15min 27s


** Data: keep all index and dates **

In [9]:
### fomulate location
def fomulate_loc(w):
    """formulate storage location according to es data"""
    w['all_rec_food']["loc"] = w['all_rec_food']["_index"] + "/" + \
    w["food"] + "/" + \
    w['all_rec_food']["_id"]

def fomulate_timestamp(w):
    if w["_index"] == 'what_to_eat':
        w['all_rec_food']["timestamp"] = w['all_rec_food']['_source.startTimestampGMT']
    else:
        d = f.retrive_from_es(w, "crawling_cycle")
        cycle_time = d.set_index(w['cycle_id']).to_dict()[w['cycleStart']]
        # remove na for food names
        w['all_rec_food'] = w['all_rec_food'][~w['all_rec_food'][w["food_name"]].isnull()]
        # assign time according to food cycle
        w['all_rec_food']["timestamp"] = w['all_rec_food'][w['food_cycle']].apply(lambda s: cycle_time[s])

In [10]:
for w in websites:
    fomulate_loc(w)     
    fomulate_timestamp(w) 
    print "%s:: number of records: %d"%(w["_index"],
                                        w['all_rec_food'].shape[0])

foodpanda:: number of records: 2032443
deliveroo:: number of records: 1459312
what_to_eat:: number of records: 226512


** Save food data with all details **

In [23]:
## save to p file
import pandas as pd
df1 = pd.concat(w['all_rec_food'] for w in websites)
file_name = "__".join(["all_food_records", f.today()]) + ".p"
f.save_file(df1, file_name)

saved: all_food_records__20180315.p


In [29]:
# ## save to json (TEMP FILE, NOT USED ANYWHERE LATER)
# df1.reset_index(inplace=True, drop=True)
# file_name = "__".join(["all_food_records", f.today()]) + ".json"
# df1.to_json(file_name, orient='index')
# print file_name

In [30]:
print df1.shape
print df1.columns.tolist()
df1.sample(2,random_state=2008)

(3718267, 37)
['_id', '_index', '_score', '_source.aggs.group_by_state.terms.field', '_source.category', '_source.currency', '_source.cycle', '_source.description', '_source.image', '_source.imageUrl', '_source.name', '_source.price', '_source.restaurant.address', '_source.restaurant.cuisine', '_source.restaurant.cycle', '_source.restaurant.maxrating', '_source.restaurant.name', '_source.restaurant.neighbourhood', '_source.restaurant.opening', '_source.restaurant.phone', '_source.restaurant.rating', '_source.restaurant.tag', '_source.size', '_source.startTimestampGMT', '_source.title', '_source.variations', '_source.vendor.cycle', '_source.vendor.vendor_address', '_source.vendor.vendor_budget', '_source.vendor.vendor_cuisines', '_source.vendor.vendor_image_url', '_source.vendor.vendor_name', '_source.vendor.vendor_rating', '_source.vendor.vendor_url', '_type', 'loc', 'timestamp']


,_id,_index,_score,_source.aggs.group_by_state.terms.field,_source.category,_source.currency,_source.cycle,_source.description,_source.image,_source.imageUrl,...,_source.vendor.vendor_address,_source.vendor.vendor_budget,_source.vendor.vendor_cuisines,_source.vendor.vendor_image_url,_source.vendor.vendor_name,_source.vendor.vendor_rating,_source.vendor.vendor_url,_type,loc,timestamp
1421554,AV2dMfH_cSlXBuMN7wFZ,foodpanda,0.0,NaN,Snacks,NaN,1.501564e+09,Served with chilli mayo,NaN,NaN,...,"10, Tebing Lane, #01-02, Sengkang, Singapore, ...",$$,"[Burgers, Pizza, Western]",https://asia-public.foodpanda.com/dynamic/prod...,Cafe Frenzie Bistro & Bar,4.5,/restaurant/s2zu/cafe-frenzie-bistro,menu_item,foodpanda/menu_item/AV2dMfH_cSlXBuMN7wFZ,2017-08-01 13:00:06.509207
2060095,27999,deliveroo,0.0,NaN,Sharing Plates,NaN,1.499105e+09,"Quinoa, Garlic",NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,food,deliveroo/food/27999,2017-07-03T17:55:01.668428


# Restaurant entities

In [90]:
%%time
restaurant_details = f.get_restaurant_details(df1)

CPU times: user 10min 55s, sys: 9.23 s, total: 11min 5s
Wall time: 10min 58s


In [92]:
restaurant_details.sample(2,random_state=2008)

,loc,timestamp,address,name,neighbourhood,opening,phone,rating,tag
1421554,foodpanda/menu_item/AV2dMfH_cSlXBuMN7wFZ,2017-08-01 13:00:06.509207,"10, Tebing Lane, #01-02, Sengkang, Singapore, ...",Cafe Frenzie Bistro & Bar,,,,4.5,"['Burgers', 'Pizza', 'Western']"
2060095,deliveroo/food/27999,2017-07-03T17:55:01.668428,"204 Telok Ayer Street, Singapore, 068640",MOJO,Chinatown,Open until 21:00,+6562200723,,Japanese


In [113]:
%%time
# merge records based on name
tup = lambda g: tuple(g)
unq = lambda g:  sorted(list(set(g)))
fn = {"loc":tup, "timestamp":tup, "address":unq, "neighbourhood":unq, "opening":unq,
     "phone":unq, "rating":unq, "tag":unq}
restaurant = restaurant_details.groupby("name").agg(fn)
# free the "name" column
restaurant.reset_index(inplace=True)
# ref dict: restaurant name : restaurant index
restaurant_ref = restaurant.set_index(["name"]).to_dict()["index"]
# update index to be "delivery_"+xxx
restaurant.reset_index(inplace=True)
restaurant["index"] = "delivery_"+restaurant["index"].apply(str)
restaurant.set_index("index", inplace=True)

CPU times: user 5.64 s, sys: 28 ms, total: 5.66 s
Wall time: 5.66 s


In [118]:
print restaurant.shape
print restaurant.columns.tolist()
restaurant.sample(2, random_state=2008)

(6845, 9)
['name', 'loc', 'tag', 'rating', 'address', 'timestamp', 'opening', 'neighbourhood', 'phone']


,name,loc,tag,rating,address,timestamp,opening,neighbourhood,phone
index,,,,,,,,,
delivery_3461,Menya Musash - Westgate,"(deliveroo/food/141830, deliveroo/food/141831,...","[Free Delivery, Japanese]",[],"[3 Gateway Drive #02-07, Singapore, 608532]","(2017-07-03T17:55:01.668428, 2017-07-03T17:55:...","[Open until 21:30, Opens at 11:30]",[Teban],[+6564659381]
delivery_1518,Don't Tell Mama,"(foodpanda/menu_item/AV1B0-jpcSlXBuMN30Ke, foo...","[['Mexican', 'Korean', 'Tacos', 'Burritos', 'C...","[4.2, 4.3]","[, 52 Tanjong Pagar Road, 088473 Singapore, 52...","(2017-07-15 00:25:05.695006, 2017-07-15 00:25:...",[],[],[]


In [119]:
## save to json
file_name = "__".join(["restaurant_entities", f.today()]) + ".json"
restaurant.to_json(file_name, orient='index')
print "saved: %s"%file_name

saved: restaurant_entities__20180316.json


In [134]:
## save to p file
file_name = "__".join(["restaurant_entities", f.today()]) + ".p"
f.save_file(restaurant, file_name)

saved: restaurant_entities__20180316.p


# Food entities

** clean name of the food **

In [123]:
%%time
food_details = f.get_food_details(df1, restaurant_ref)

CPU times: user 7min 54s, sys: 5.49 s, total: 8min
Wall time: 7min 55s


In [124]:
print food_details.shape
print food_details.columns.tolist()
food_details.sample(2, random_state=2008)

(3718267, 7)
['loc', 'timestamp', 'desc', 'food_name', 'price', 'tag', 'vendors']


,loc,timestamp,desc,food_name,price,tag,vendors
1421554,foodpanda/menu_item/AV2dMfH_cSlXBuMN7wFZ,2017-08-01 13:00:06.509207,Served with chilli mayo,Popcorn Chicken,"[{'price': 14.34, 'title': 'default'}]",Snacks,delivery_862
2060095,deliveroo/food/27999,2017-07-03T17:55:01.668428,"Quinoa, Garlic",Foie Gras Chahan,19.26,Sharing Plates,delivery_3290


In [127]:
# ref dict: food_name: clean_name
food_names = set(food_details["food_name"].tolist())
print "with %d unique menu names in %d records"%(len(food_names), food_details.shape[0])
clean_food_ref = {s:f.clean_name_v2(s) for s in food_names}

with 200779 unique menu names in 3718267 records


In [136]:
%%time
# merge records based on name
tup = lambda g: tuple(g)
unq = lambda g:  sorted(list(set(g)))
fn = {"food_name":f.count_freq, "vendors": unq, "desc":unq, "price":tup, 
      "tag":unq, "loc":unq, "timestamp":unq}
food_details["clean_name"] = food_details["food_name"].apply(lambda s: clean_food_ref[s])
food_items = food_details.groupby("clean_name").agg(fn)
# free the "clean_name" column
food_items.reset_index(inplace=True)
# # ref dict: restaurant name : restaurant index
# restaurant_ref = restaurant.set_index(["name"]).to_dict()["index"]
# update index to be "food_"+xxx
food_items.reset_index(inplace=True)
food_items["index"] = "food_"+food_items["index"].apply(str)
food_items.set_index("index", inplace=True)

CPU times: user 46.4 s, sys: 476 ms, total: 46.9 s
Wall time: 46.5 s


In [137]:
print food_items.shape
print food_items.columns.tolist()
food_items.sample(2, random_state=2008)

(139780, 8)
['clean_name', 'loc', 'food_name', 'timestamp', 'price', 'vendors', 'tag', 'desc']


,clean_name,loc,food_name,timestamp,price,vendors,tag,desc
index,,,,,,,,
food_130659,toro cha shao persons,"[deliveroo/food/19109, deliveroo/food/AV06uSiz...","[(Toro Cha Shao 2-3 persons, 5)]","[2017-07-03T17:55:01.668428, 2017-07-13T14:37:...","(16.5, 16.5, 16.5, 16.5, 16.5)",[delivery_5687],[Chef's Recommendation Roast],[]
food_127787,the guanyin,"[deliveroo/food/AV2ZY3TUcSlXBuMN4ZOF, delivero...","[(The Guanyin, 172)]","[2017-08-01T00:00:04.909498, 2017-09-01 01:00:...","([{'price': 4.71, 'title': 'default'}], [{'pri...","[delivery_3921, delivery_3922, delivery_3923, ...","[Kopi - Teh, Kopi Teh, Kopi and Teh, Kopi-Teh,...","[, Currently sold out, Default, Per serving.]"


** construct test case **

In [138]:
# save test case to csv
file_name = "001_entity_identification_test_case_" + f.today()+".csv"
test_case = food_items.sample(100, random_state=2008)[["food_name", "clean_name"]]
test_case["food_name"] = test_case["food_name"].apply(lambda s: s[0][0])
test_case.to_csv(file_name,index=False)
print "saved: %s"%file_name

saved: 001_entity_identification_test_case_20180316.csv


In [139]:
## save to json
file_name = "__".join(["food_entities", f.today()]) + ".json"
food_items.to_json(file_name, orient='index')
print "saved: %s"%file_name

saved: food_entities__20180316.json


In [140]:
## save to p file
file_name = "__".join(["food_entities", f.today()]) + ".p"
f.save_file(food_items, file_name)

saved: food_entities__20180316.p
